#  **LLM DETECT AI GENERATED**


Importing the required libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# DATA PREPROCESSING

### We largely depend on nltk tools for data-preprocessing

To begin with we are going to define methods for cleaning text(clean_text) where we remove HTML tags, non-alphabetic characters (except whitespace), and convert the text to lowercase.
   Also tokenize_sentences and tokenize_words for the tokenization of text data into sentences and words.At the end removing the stop words.
  

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    return text

def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

def tokenize_words(sentence):
    words = word_tokenize(sentence)
    return words

def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words

In [ ]:
def preprocess_text(text):
    cleaned_text = clean_text(text)
    sentences = tokenize_sentences(cleaned_text)
    words = tokenize_words(sentences[0])
    filtered_words = remove_stopwords(words)

    return filtered_words



# Loading the data into the pandas dataframe
  Dataset comprises of train_essays from the competition
     argupt essays(totally ai-generated), and most often used dataset in the competition train_drcat_v2 by DAIgt.
     

In [ ]:
# load the data to a pandas dataframe
train_data= pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

In [ ]:
train_data=train_data.drop(columns=['prompt_id'])

In [ ]:
new_3=pd.read_csv('/kaggle/input/argugpt/argugpt.csv')
new_3['generated']=1

In [ ]:
new_3=new_3.drop(columns=['prompt_id','prompt','model','temperature','exam_type','score','score_level'])

In [ ]:
new_3

,id,text,generated
0,weccl_30,There are many people who think that universit...,1
1,weccl_51,There are a number of reasons why people might...,1
2,weccl_48,There are many reasons why university educatio...,1
3,weccl_50,There are many people who think that universit...,1
4,weccl_55,There is a general consensus that university e...,1
...,...,...,...
4033,gre_514,The notion that one must be forced to defend a...,1
4034,gre_1107,I strongly agree with the statement that menta...,1
4035,gre_1163,"In today’s world, where competition is highly ...",1
4036,gre_1174,Education is one of the most powerful tools th...,1


In [ ]:
train_data=pd.concat([train_data,new_3],ignore_index=True)

In [ ]:
train_data.columns = ['id', 'text','generated']
train_data

,id,text,generated
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
5411,gre_514,The notion that one must be forced to defend a...,1
5412,gre_1107,I strongly agree with the statement that menta...,1
5413,gre_1163,"In today’s world, where competition is highly ...",1
5414,gre_1174,Education is one of the most powerful tools th...,1


In [ ]:
new=pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
new

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
...,...,...,...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44864,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44865,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44866,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True


 filtering the dataset by RDizzl3_seven came in handy and helped me improve my public score

In [ ]:
new= new[new.RDizzl3_seven]
new=new.reset_index(drop=True)

In [ ]:
new=new.drop(columns=['prompt_name','source','RDizzl3_seven'])
new

,text,label
0,Cars have been around for awhile and they have...,0
1,Have you ever thought what it would be like no...,0
2,What you are about to read is going to give yo...,0
3,cars have many flaws nd and in this day and ag...,0
4,There are many advantages of limiting car usag...,0
...,...,...
20445,"Dear Senator,\n\nI am writing to you today to ...",1
20446,"Dear Senator,\n\nI am writing to you today to ...",1
20447,"Dear Senator,\n\nI am writing to you today to ...",1
20448,"Dear Senator,\n\nI am writing to you today to ...",1


In [ ]:
new.columns=['text','generated']

In [ ]:
new['id']=np.random.rand(20450)
new

,text,generated,id
0,Cars have been around for awhile and they have...,0,0.222362
1,Have you ever thought what it would be like no...,0,0.340333
2,What you are about to read is going to give yo...,0,0.455175
3,cars have many flaws nd and in this day and ag...,0,0.118815
4,There are many advantages of limiting car usag...,0,0.066707
...,...,...,...
20445,"Dear Senator,\n\nI am writing to you today to ...",1,0.518218
20446,"Dear Senator,\n\nI am writing to you today to ...",1,0.816804
20447,"Dear Senator,\n\nI am writing to you today to ...",1,0.676522
20448,"Dear Senator,\n\nI am writing to you today to ...",1,0.445930


In [ ]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20450 entries, 0 to 20449
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       20450 non-null  object 
 1   generated  20450 non-null  int64  
 2   id         20450 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 479.4+ KB


In [ ]:
new.generated.value_counts()

generated
0    14250
1     6200
Name: count, dtype: int64

In [ ]:
train_data=pd.concat([train_data,new],ignore_index=True)
train_data

,id,text,generated
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
25861,0.518218,"Dear Senator,\n\nI am writing to you today to ...",1
25862,0.816804,"Dear Senator,\n\nI am writing to you today to ...",1
25863,0.676522,"Dear Senator,\n\nI am writing to you today to ...",1
25864,0.44593,"Dear Senator,\n\nI am writing to you today to ...",1


In [ ]:
# checking for missing values
train_data.isnull().sum()

id           0
text         0
generated    0
dtype: int64

In [ ]:
# replacing the missing values with null string
train_data = train_data.fillna('')

In [ ]:
train_data=train_data.drop_duplicates(subset='text')
train_data

,id,text,generated
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
25861,0.518218,"Dear Senator,\n\nI am writing to you today to ...",1
25862,0.816804,"Dear Senator,\n\nI am writing to you today to ...",1
25863,0.676522,"Dear Senator,\n\nI am writing to you today to ...",1
25864,0.44593,"Dear Senator,\n\nI am writing to you today to ...",1


In [ ]:
train_data=train_data.reset_index(drop=True)
train_data

,id,text,generated
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
24483,0.518218,"Dear Senator,\n\nI am writing to you today to ...",1
24484,0.816804,"Dear Senator,\n\nI am writing to you today to ...",1
24485,0.676522,"Dear Senator,\n\nI am writing to you today to ...",1
24486,0.44593,"Dear Senator,\n\nI am writing to you today to ...",1


In [ ]:
train_data['generated'].value_counts()

generated
0    14250
1    10238
Name: count, dtype: int64

### Hence we are finally equipped with 24488 essays of which 14,250 are human written and 10238 are ai-generated

# Applying Datapreprocessing to Text-data

In [ ]:
train_data['text'] = train_data['text'].apply(lambda x: preprocess_text(x))
train_data

,id,text,generated
0,0059830c,"[cars, cars, around, since, became, famous, he...",0
1,005db917,"[transportation, large, necessity, countries, ...",0
2,008f63e3,"[americas, love, affair, vehicles, seems, cool...",0
3,00940276,"[often, ride, car, drive, one, motor, vehicle,...",0
4,00c39458,"[cars, wonderful, thing, perhaps, one, worlds,...",0
...,...,...,...
24483,0.518218,"[dear, senator, writing, today, express, views...",1
24484,0.816804,"[dear, senator, writing, today, express, suppo...",1
24485,0.676522,"[dear, senator, writing, today, express, stron...",1
24486,0.44593,"[dear, senator, writing, today, express, stron...",1


In [ ]:
train_data['text'] = train_data['text'].apply(lambda x: ' '.join(x))
X = train_data['text'].tolist()

In [ ]:
Y = train_data['generated'].values

To convert the Textual data into feature vectors we are using TfidfVectorizer because of its ability to convert raw data into numerical vectors sutable for analysis.

In [ ]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
X

<24488x72566 sparse matrix of type '<class 'numpy.float64'>'
	with 2914903 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42
)

# Model Training

The models that are used here are


1.   SGD-classifier-It optimizes the model parameters by iteratively updating them with gradients computed on small batches of training data.it supports various Loss functions

2.   MultinomialNB-works on the principle of Bayes theorem and assumes that the features are conditionally independent given the class variable.

Voting-classifer  - is a machine learning model that trains on an ensemble of numerous models and predicts an output by computing the average predicted probabilities (soft voting) from its constituent models.


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

#lr_model = LogisticRegression()
mnb_model=MultinomialNB(alpha=0.0225)
sgd_model = SGDClassifier(max_iter=8000, loss="modified_huber", random_state=42)
sgd_model2 = SGDClassifier(max_iter=12000, tol=5e-4, loss="modified_huber", class_weight="balanced")
sgd_model3 = SGDClassifier(max_iter=15000, tol=3e-4, loss="modified_huber", early_stopping=True)

#ensemble = VotingClassifier(estimators=[('lr', lr_model),
                                        #('sgd', sgd_model),
                                        #('sgd2', sgd_model2),
                                        #('sgd3', sgd_model3),
                                        #('mnb',mnb_model)
                                       #],
                            #weights=[0.01,0.20,0.01,0.20,0.31],
                            #voting='soft'
                           #)
ensemble = VotingClassifier(estimators=[('sgd', sgd_model),
                                        ('mnb',mnb_model)
                                       ],
                            weights=[0.59,0.41],
                            voting='soft'
                           )
ensemble.fit(X_train, y_train)


VotingClassifier(estimators=[('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, random_state=42)),
                             ('mnb', MultinomialNB(alpha=0.0225))],
                 voting='soft', weights=[0.59, 0.41])

In [ ]:
y_pred=ensemble.predict(X_test)

print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

predicted_probabilities = ensemble.predict_proba(X_test)[:, 1]
print(predicted_probabilities)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4651
           1       1.00      0.99      0.99      3431

    accuracy                           0.99      8082
   macro avg       0.99      0.99      0.99      8082
weighted avg       0.99      0.99      0.99      8082

0.9933184855233853
[1.35247088e-07 9.77915574e-01 8.65231791e-01 ... 6.01380386e-05
 9.99182663e-01 1.03963226e-04]


# Prediciting Test dataset using trained model

In [ ]:
test_essays=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_essays

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [ ]:
sub=test_essays.drop(columns=['prompt_id','text'])
X_newtest=test_essays.text

In [ ]:
X_newtest

0    Aaa bbb ccc.
1    Bbb ccc ddd.
2    CCC ddd eee.
Name: text, dtype: object

In [ ]:
test_essays['text'] = test_essays['text'].apply(lambda x: preprocess_text(x))

In [ ]:
test_essays['text'] = test_essays['text'].apply(lambda x: ' '.join(x))

In [ ]:
X_newtest=test_essays['text'].values

In [ ]:
X_newtest = vectorizer.transform(X_newtest)

In [ ]:
y_pred = ensemble.predict(X_newtest)

predicted_probabilities = ensemble.predict_proba(X_newtest)[:, 1]
print(predicted_probabilities)

[0.79094795 0.5552065  0.5552065 ]


In [ ]:
sub=test_essays.drop(columns=['prompt_id','text'])
sub['generated']=predicted_probabilities
sub.to_csv('submission.csv',index=False)

On submitting this prediction I have got the public score of 0.869

Initially without datapreprocessing i have used the tf-idf vectoriser and models individually buts got score of 0.81

After data prepocessing and fine tuning the  parameters and models i have got the score of 0.869

Here is the code which was initially there during the finetuning

In [ ]:
'''
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

lr_model = LogisticRegression()
mnb_model=MultinomialNB(alpha=0.0225)
sgd_model = SGDClassifier(max_iter=8000, loss="modified_huber", random_state=42)
sgd_model2 = SGDClassifier(max_iter=12000, tol=5e-4, loss="modified_huber", class_weight="balanced")
sgd_model3 = SGDClassifier(max_iter=15000, tol=3e-4, loss="modified_huber", early_stopping=True)

ensemble = VotingClassifier(estimators=[('lr', lr_model),
                                        ('sgd', sgd_model),
                                        ('sgd2', sgd_model2),
                                        ('sgd3', sgd_model3),
                                        ('mnb',mnb_model)
                                       ],
                            weights=[0.20,0.20,0.20,0.20,0.20],
                            voting='soft'
                           )
'''

But finally i ended up with Sgd and mnb with weights 0.59 and 0.41